In [8]:
from google.colab import files
import pandas as pd
import io
 
uploaded = files.upload()
normal_df = pd.read_csv(io.BytesIO(uploaded['Normal_ds.csv']))
uploaded = files.upload()
attack_df = pd.read_csv(io.BytesIO(uploaded['Attack_ds.csv']))

Saving Normal_ds.csv to Normal_ds (4).csv


Saving Attack_ds.csv to Attack_ds.csv


In [9]:
print(attack_df)

    Nodes  Distance  PLR  DIO sent all  DIO Recv all  DIOsentAvg  DIOrecvAvg  \
0      10        20   10          1026         15336         104        1057   
1      10        30   10          1060          7780         106        1063   
2      10        40   10          1044          5464         104        1044   
3      20        20   10          2052         30224         102        1024   
4      20        30   10          2016         16152         100        1000   
5      20        40   10          1963         10768          98         985   
6      40        20   10          3993         57043          99         991   
7      40        40    5          3090         12600          77         798   
8      50        20   15          3900         42900          78         787   
9      50        35    5          3862         23625          77         778   
10     60        20   10          4680         51480          78         803   
11     60        45   18          4680  

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
from sklearn.preprocessing import MinMaxScaler



normal_df.fillna(normal_df.median())
attack_df.fillna(attack_df.median())


# Split the normal dataset into training and testing sets
normal_train_df, normal_test_df = train_test_split(normal_df, test_size=0.2, random_state=42)

# Combine the testing sets of both datasets
test_df = pd.concat([normal_test_df, attack_df])

# Prepare the training data
X_train = normal_train_df.drop('IsAttack', axis=1).values
X_train = normal_train_df.drop('Attack', axis=1).values
y_train = normal_train_df['IsAttack'].values

# Create a PCA object with 2 components
'''pca = PCA(n_components=2)

# Fit the PCA object to your original training data
pca.fit(X_train)

# Transform your original training data using the PCA object
X_train_pca = pca.transform(X_train)

# Update your training data with the transformed data
X_train = X_train_pca'''


# Train a one-class SVM on the training data
clf = OneClassSVM(nu=0.000001, kernel='rbf', gamma='auto').fit(X_train)

# Prepare the testing data and labels
X_test = test_df.drop('IsAttack', axis=1).values
y_true = test_df['IsAttack'].values
X_test = test_df.drop('Attack', axis=1).values

'''X_test_pca = pca.transform(X_test)

# Update your training data with the transformed data
X_test = X_test_pca'''


# Predict the labels for the testing data
y_pred = clf.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_true, y_pred)
conf_mat = confusion_matrix(y_true, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Confusion matrix:\n{conf_mat}")


Accuracy: 0.7058823529411765
Confusion matrix:
[[48  0]
 [20  0]]


<ipython-input-21-aee9e69c290e>:13: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  normal_df.fillna(normal_df.median())
<ipython-input-21-aee9e69c290e>:14: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  attack_df.fillna(attack_df.median())


In [31]:
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.preprocessing import StandardScaler
import numpy as np

# Create a training set that consists only of normal data

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Define the autoencoder architecture
input_dim = X_train.shape[1]
encoding_dim = 10
hidden_size = 10

input_layer = Input(shape=(input_dim,))
hidden_layer1 = Dense(hidden_size, activation="relu")(input_layer)
hidden_layer2 = Dense(hidden_size, activation="relu")(hidden_layer1)
encoder = Dense(encoding_dim, activation="relu")(hidden_layer2)
hidden_layer3 = Dense(hidden_size, activation="relu")(encoder)
hidden_layer4 = Dense(hidden_size, activation="relu")(hidden_layer3)
decoder = Dense(input_dim, activation="sigmoid")(hidden_layer4)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer="adam", loss="mean_squared_error")

# Train the autoencoder on the normal data
autoencoder.fit(X_train, X_train, epochs=200, batch_size=32, shuffle=True, validation_split=0.1)

# Encode the normal data
encoder_layer = autoencoder.layers[1]
encoder_model = Model(inputs=input_layer, outputs=encoder_layer.output)
encoded_data = encoder_model.predict(X_train)

# Compute the reconstruction error
decoded_data = autoencoder.predict(X_train)
mse = np.mean(np.power(X_train - decoded_data, 2), axis=1)

# Set a threshold for anomaly detection
threshold = np.percentile(mse, 95)

# Evaluate the model on a test set
X_test = scaler.transform(X_test)
decoded_test = autoencoder.predict(X_test)
mse_test = np.mean(np.power(X_test - decoded_test, 2), axis=1)
y_pred = (mse_test > threshold).astype(int)
y_pred = 2*y_pred - 1




# Evaluate the performance of the classifier
accuracy = accuracy_score(y_true, y_pred)
conf_mat = confusion_matrix(y_true, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Confusion matrix:\n{conf_mat}")



Epoch 1/200
6/6 [==============================] - 1s 32ms/step - loss: 1.1081 - val_loss: 0.8794
Epoch 2/200
6/6 [==============================] - 0s 10ms/step - loss: 1.1029 - val_loss: 0.8762
Epoch 3/200
6/6 [==============================] - 0s 7ms/step - loss: 1.0977 - val_loss: 0.8725
Epoch 4/200
6/6 [==============================] - 0s 8ms/step - loss: 1.0919 - val_loss: 0.8683
Epoch 5/200
6/6 [==============================] - 0s 7ms/step - loss: 1.0848 - val_loss: 0.8634
Epoch 6/200
6/6 [==============================] - 0s 7ms/step - loss: 1.0757 - val_loss: 0.8572
Epoch 7/200
6/6 [==============================] - 0s 8ms/step - loss: 1.0648 - val_loss: 0.8491
Epoch 8/200
6/6 [==============================] - 0s 7ms/step - loss: 1.0503 - val_loss: 0.8384
Epoch 9/200
6/6 [==============================] - 0s 8ms/step - loss: 1.0315 - val_loss: 0.8247
Epoch 10/200
6/6 [==============================] - 0s 8ms/step - loss: 1.0075 - val_loss: 0.8068
Epoch 11/200
6/6 [=========